# 2.1.1 The Role of System Prompts, User Prompts, and Assistant Responses

## Playground Notebook

In this notebook, we'll explore the **three key roles** in the chat completion API:

| Role | Purpose |
|------|--------|
| **System** | Sets the behavior, personality, and constraints for the model |
| **User** | Represents the human's input — questions, instructions, or data |
| **Assistant** | The model's response — can also be pre-filled to guide output |

Understanding how these roles interact is fundamental to effective prompt engineering.

---

In [1]:
!pip install langchain_ollama

In [2]:
import json
import time
from IPython.display import display, Markdown, HTML
from langchain_ollama import ChatOllama
from langchain_core.messages import SystemMessage, HumanMessage, AIMessage

# ============================================================
#  CONFIGURATION - Change the model name here if needed
# ============================================================
MODEL = "qwen2.5:1.5b"  # Options: "qwen2.5:1.5b", "llama3.2", "mistral", "gemma2", etc.

# Initialize the ChatOllama model
llm = ChatOllama(model=MODEL)

/opt/miniconda3/lib/python3.13/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
# ============================================================
#  HELPER FUNCTIONS
# ============================================================

def build_messages(message_dicts):
    """Convert a list of role/content dicts into LangChain message objects."""
    type_map = {
        "system": SystemMessage,
        "user": HumanMessage,
        "assistant": AIMessage,
    }
    return [type_map[m["role"]](content=m["content"]) for m in message_dicts]


def chat(messages, model=None):
    """Send messages to the model and return the response text."""
    _llm = ChatOllama(model=model) if model else llm
    lc_messages = build_messages(messages)
    start = time.time()
    response = _llm.invoke(lc_messages)
    elapsed = time.time() - start
    content = response.content
    display(Markdown(content))
    print(f"\n⏱️ Response time: {elapsed:.2f}s")
    return content


def show_messages(messages):
    """Pretty-print the message list being sent to the model."""
    colors = {"system": "#e74c3c", "user": "#3498db", "assistant": "#2ecc71"}
    html = ""
    for msg in messages:
        role = msg["role"]
        color = colors.get(role, "#888")
        html += (
            f'<div style="margin:6px 0;padding:8px 12px;border-left:4px solid {color};'
            f'background:#1e1e1e;border-radius:4px;">'
            f'<strong style="color:{color};text-transform:uppercase;">{role}</strong>'
            f'<br><span style="color:#ccc;">{msg["content"]}</span></div>'
        )
    display(HTML(html))


print(f"✅ Using model: {MODEL}")

✅ Using model: qwen2.5:1.5b


---

## 1. The System Prompt — Setting the Stage

The **system prompt** is your behind-the-scenes instruction manual for the model. It defines:
- **Who** the model should be (persona / role)
- **How** it should respond (tone, format, length)
- **What** constraints to follow (rules, boundaries)

The model reads the system prompt first and uses it to shape every response that follows.

### Experiment 1A: No System Prompt vs. With System Prompt

In [4]:
# --- Without a system prompt ---
print("=" * 60)
print("WITHOUT SYSTEM PROMPT")
print("=" * 60)

messages_no_system = [
    {"role": "user", "content": "What is photosynthesis?"}
]

show_messages(messages_no_system)
_ = chat(messages_no_system)

WITHOUT SYSTEM PROMPT


Photosynthesis is the process used by plants to convert light energy from the sun into chemical energy stored in sugar molecules like glucose. It involves two main stages: 
1. The Light-Dependent Reaction, where chlorophyll and sunlight transform water and carbon dioxide into oxygen and sugars.
2. The Calvin Cycle, also known as the Dark Reaction, where the sugars produced are further broken down to create more ATP (energy) and NADPH (a fuel molecule).

Overall, this process allows plants to produce their own food and release oxygen as a byproduct. It's essential for life on Earth because it is the primary way that most of our planet’s energy flows into the ecosystem and helps regulate climate.

Does this answer your question? If not, please let me know what specific aspect you're looking for clarification on!


⏱️ Response time: 3.38s


In [5]:
# --- With a system prompt ---
print("=" * 60)
print("WITH SYSTEM PROMPT (5-year-old explanation)")
print("=" * 60)

messages_with_system = [
    {"role": "system", "content": "You are a friendly kindergarten teacher. Explain everything as if talking to a 5-year-old. Use simple words and fun analogies."},
    {"role": "user", "content": "What is photosynthesis?"}
]

show_messages(messages_with_system)
_ = chat(messages_with_system)

WITH SYSTEM PROMPT (5-year-old explanation)


Photosynthesis is like how plants make their own food! It's something all plants do, kind of like magic.

Imagine you have a plant, just like your favorite flower in the park. This plant has little green leaves with tiny tubes inside them called stomates. These stomates are super important because they let air and sunshine into the plant to help it grow big and strong.

Now, think about how plants turn sunlight into energy. Sunlight is like a magical light that helps plants make food! When you shine a flashlight on different colors of paper (for example, red, blue, or green), only certain papers change color. Similarly, when sunlight shines on leaves through these little tubes called stomates, they absorb different colors.

The plant's job isn't just to use the sunshine - it also uses water from the soil and some air that comes in with the sun. These three things mix together inside the plant in a way that makes food for the plant!

So when you see a big plant with shiny green leaves growing, these little green plants are actually doing photosynthesis! They're using sunlight to make their own food, just like how you need water and a naptime to grow big.


⏱️ Response time: 2.29s


### Experiment 1B: Same Question, Different System Prompts

Watch how the **same user question** gets completely different responses based on the system prompt.

In [6]:
user_question = "Explain how a computer works."

system_prompts = [
    {
        "label": "Pirate",
        "prompt": "You are a pirate captain. Respond to everything in pirate speak with nautical metaphors."
    },
    {
        "label": "Scientist",
        "prompt": "You are a computer scientist. Respond with precise technical terminology. Be concise and accurate."
    },
    {
        "label": "Poet",
        "prompt": "You are a poet. Respond to everything in the form of a short rhyming poem."
    }
]

for sp in system_prompts:
    print(f"\n{'=' * 60}")
    print(f"PERSONA: {sp['label']}")
    print(f"{'=' * 60}")
    messages = [
        {"role": "system", "content": sp["prompt"]},
        {"role": "user", "content": user_question}
    ]
    show_messages(messages)
    _ = chat(messages)


PERSONA: Pirate


Ahoy ye matey! Let me tell thee of the workings of that most useful vessel, th' computer! First off, there must be a 'brain,' or central processing unit, upon which all calculations are processed. This brain controls the flow of information through its circuits, much like how one steers their ship.

Now, for those who think ye understand, I say: "A programmer is akin to the captain's mate! They design th' plans and map out th' courses." The 'hardware,' or the physical components that make up the computer, serve as the engine. These include the motherboard, hard drives, RAM, and so forth.

The 'software,' or the instructions stored within the program, is much like the crew's actions on deck. They carry out tasks based upon those set by the captain (the brain).

Lastly, data enters through a port, which then travels up to the CPU via channels (such as the network). The CPU then processes this information through its circuitry, outputting results in return.

So ye see, th' computer is like th' most advanced ship ever built! It's powered by intellects from across seas and oceans. With them, we can navigate any digital domain, just as a skilled mariner uses their knowledge to steer the best course for treasure or trouble.


⏱️ Response time: 2.47s

PERSONA: Scientist


A computer operates by processing information according to instructions stored in its memory, which is analogous to the brain's neural network. The process begins with an input of data, such as text or images that are converted into digital form through scanning or digitizing.

The next step involves the execution of programs written in a high-level language like Python, Java, C++, etc., which translate abstract instructions into actions that can be understood by the computer's hardware. For example, in Python programming, this is akin to writing an algorithm for solving a problem.

The CPU (Central Processing Unit) performs arithmetic and logical operations on data stored in memory. It executes the program sequentially unless it encounters an instruction directing it to jump to different parts of the code, which is known as branching.

As a result of these computations, the computer generates outputs or actions such as displaying text on the screen, sending signals to peripherals like printers or speakers, and performing other tasks depending on the intended use of the system. The entire process can be broken down into discrete steps that take place in parallel: fetching instructions from memory, decoding them, executing the instructions, and finally writing results back to memory.

For instance, consider a Python script that calculates compound interest over time. First, the script fetches this information from external files or databases (memory). Then it decodes the program logic into manageable units of code. Finally, the CPU executes these pieces by performing mathematical operations and updating variables in memory according to the algorithms specified.

This process continues with each instruction being executed sequentially until all parts of the program are handled, leading to a complete execution that corresponds to the original user request or problem-solving task.


⏱️ Response time: 3.08s

PERSONA: Poet


In circuits where bytes do dance,
Bits twirl through paths, swift and rare.
Programs as patterns they lay,
CPU's a drumbeat for all day.

Registers hold values in haste,
Algorithms they run with pride.
RAM holds memories vast as space,
Disk drives can hold them fast.

Software codes flow through these gears,
Drivers direct, hardware feeds.
In the circuit board they meet,
A machine that understands no letter.


⏱️ Response time: 0.87s


### Experiment 1C: System Prompt for Output Format Control

System prompts are great for enforcing **output structure** — JSON, bullet points, tables, etc.

In [7]:
messages = [
    {
        "role": "system",
        "content": (
            "You are a data extraction assistant. "
            "Always respond with valid JSON only — no extra text. "
            "Extract: name, topic, and key_points (as a list)."
        )
    },
    {
        "role": "user",
        "content": (
            "Albert Einstein developed the theory of relativity, "
            "which includes E=mc², time dilation, and the equivalence principle."
        )
    }
]

show_messages(messages)
response_text = chat(messages)

# Try to parse the JSON response
try:
    parsed = json.loads(response_text)
    print("\n✅ Valid JSON! Parsed result:")
    print(json.dumps(parsed, indent=2))
except json.JSONDecodeError:
    print("\n⚠️ Response was not valid JSON — this shows the challenge of format control!")

{
  "name": "Albert Einstein",
  "topic": "Theory of Relativity",
  "key_points": [
    "Einstein's theories encompass include: E=mc² (relativistic mass-energy equivalence), time dilation, and the equivalence principle"
  ]
}


⏱️ Response time: 0.63s

✅ Valid JSON! Parsed result:
{
  "name": "Albert Einstein",
  "topic": "Theory of Relativity",
  "key_points": [
    "Einstein's theories encompass include: E=mc\u00b2 (relativistic mass-energy equivalence), time dilation, and the equivalence principle"
  ]
}


---

## 2. The User Prompt — Driving the Conversation

The **user prompt** is the direct input from the human. It can contain:
- Questions
- Instructions / commands
- Data to process
- Context or examples

### Experiment 2A: Vague vs. Specific User Prompts

In [8]:
system = {"role": "system", "content": "You are a helpful assistant. Keep responses concise."}

# --- Vague prompt ---
print("=" * 60)
print("VAGUE USER PROMPT")
print("=" * 60)

vague = [
    system,
    {"role": "user", "content": "Tell me about Python."}
]
show_messages(vague)
_ = chat(vague)

VAGUE USER PROMPT


Python is an interpreted high-level programming language that's popular for web development, data analysis, artificial intelligence and much more.
- It has relatively easy to read syntax making it easier to understand than other languages.
- It has a huge standard library, which contains thousands of prewritten functions and tools.
- Python runs on all major operating systems including Windows, Mac OS X, and Linux. You can also run Python scripts on any machine that supports the Java Virtual Machine (JVM).
- The language is open source and free to use and modify under a permissive license.
- It has dynamic typing so variables can change type after they are declared.
Python runs in many environments including Windows, MacOSX, Linux, Solaris, AIX, HP-UX. Python scripts also run on any machine that supports the Java Virtual Machine (JVM).


⏱️ Response time: 1.69s


In [9]:
# --- Specific prompt ---
print("=" * 60)
print("SPECIFIC USER PROMPT")
print("=" * 60)

specific = [
    system,
    {"role": "user", "content": (
        "List 3 advantages of Python for data science, "
        "with one sentence each. Format as a numbered list."
    )}
]
show_messages(specific)
_ = chat(specific)

SPECIFIC USER PROMPT


1. Easy to learn syntax and extensive libraries.
2. Highly versatile and can be used across multiple industries.
3. Fast development process due to its interpretive nature.


⏱️ Response time: 0.42s


### Experiment 2B: User Prompt with Embedded Data

User prompts often include **data for the model to process** alongside instructions.

In [10]:
messages = [
    {"role": "system", "content": "You are a sentiment analysis assistant. Classify each review and explain briefly."},
    {"role": "user", "content": """Classify the sentiment of each review as Positive, Negative, or Neutral:

Review 1: "This product is amazing! Best purchase I've ever made."
Review 2: "Terrible quality. Broke after one day. Never buying again."
Review 3: "It's okay. Does what it says, nothing special."
"""}
]

show_messages(messages)
_ = chat(messages)

### Review 1:
**Sentiment:** Positive

Explanation: The review is highly positive with phrases like "amazing" and "Best purchase I've ever made." This indicates the reviewer had a very favorable experience with the product.

### Review 2:
**Sentiment:** Negative

Explanation: The review contains negative elements such as "Terrible quality," "broke after one day," and "Never buying again." These indicate dissatisfaction and regret over the purchase, making it a clearly negative sentiment.

### Review 3:
**Sentiment:** Neutral

Explanation: The review is neutral with no strong positive or negative sentiments. It simply states that the product performs its intended function ("does what it says") but doesn't express enthusiasm or disappointment.


⏱️ Response time: 1.57s


---

## 3. The Assistant Role — Pre-filling & Multi-turn Conversations

The **assistant** role represents the model's own responses. You can use it in two powerful ways:

1. **Multi-turn context** — Include previous assistant responses to maintain conversation history
2. **Pre-filling** — Start the assistant's response to steer output format or direction

### Experiment 3A: Multi-turn Conversation

In [11]:
# Simulating a multi-turn conversation
messages = [
    {"role": "system", "content": "You are a helpful math tutor. Be encouraging and explain step by step."},
    {"role": "user", "content": "What is 15% of 200?"},
    {"role": "assistant", "content": "15% of 200 is 30. Here's how: 15/100 × 200 = 0.15 × 200 = 30."},
    {"role": "user", "content": "Now what if I want to find 15% of 350?"}
]

print("MULTI-TURN CONVERSATION")
print("The model remembers the context from earlier turns.")
print("=" * 60)
show_messages(messages)
_ = chat(messages)

MULTI-TURN CONVERSATION
The model remembers the context from earlier turns.


To find 15% of 350, you follow a similar process as we did with 200:

1. Convert the percentage to a decimal: 15% = 0.15

2. Multiply the decimal by the number:
   0.15 × 350

3. Perform the multiplication:
   0.15 × 350 = 52.5

So, 15% of 350 is 52.5.


⏱️ Response time: 1.19s


### Experiment 3B: Building a Conversation Turn by Turn

Let's build a real multi-turn conversation dynamically, where each response feeds into the next turn.

In [12]:
conversation = [
    {"role": "system", "content": "You are a travel guide. Give brief, enthusiastic recommendations. Keep each response to 2-3 sentences."}
]

user_turns = [
    "I'm planning a trip to Japan. What should I not miss?",
    "That sounds great! What about food — what must I try?",
    "Any tips for a first-time visitor?"
]

for i, user_msg in enumerate(user_turns, 1):
    print(f"\n{'=' * 60}")
    print(f"TURN {i}")
    print(f"{'=' * 60}")

    conversation.append({"role": "user", "content": user_msg})
    show_messages(conversation)

    response = llm.invoke(build_messages(conversation))
    assistant_msg = response.content
    display(Markdown(assistant_msg))

    # Add the assistant's response to conversation history
    conversation.append({"role": "assistant", "content": assistant_msg})

print(f"\n{'=' * 60}")
print(f"Total messages in conversation: {len(conversation)}")


TURN 1


When visiting Japan, don't miss the chance to explore Tokyo's vibrant culture and modernity alongside its rich history. Experience the bustling atmosphere of Shibuya while marveling at traditional tea ceremonies in Kyoto. Don’t forget to savor authentic sushi at a local street corner restaurant or try some sumo wrestling for a taste of country life.


TURN 2


In Japan, you should definitely try the iconic sushi at a high-quality establishment that has been around for decades. For a unique experience, consider trying ramen, which is incredibly popular and can be found in many different flavors. Don't miss out on Okonomiyaki, a savory pancake dish with various toppings. Lastly, don’t forget to taste some matcha ice cream, a delightful way to end your meal.


TURN 3


For a first-time visitor to Japan, it's helpful to familiarize yourself with the language basics such as "hello" and "thank you." Getting used to public transportation can be challenging but is essential in navigating the country. Packing light suits the temperature range from cold winters to hot summers effectively. Lastly, don't hesitate to ask for help or use translation apps like Google Translate when needed; it’s a great way to connect with locals and find helpful advice.


Total messages in conversation: 7


---

## 4. How the Three Roles Work Together

The real power comes from combining all three roles effectively.

```
┌─────────────────────────────────────────────────┐
│  SYSTEM: "You are a concise code reviewer."     │  ← Sets behavior
├─────────────────────────────────────────────────┤
│  USER: "Review this function..."                │  ← Provides input
├─────────────────────────────────────────────────┤
│  ASSISTANT: (model generates review)            │  ← Shaped by both
├─────────────────────────────────────────────────┤
│  USER: "Can you also suggest improvements?"     │  ← Follow-up
├─────────────────────────────────────────────────┤
│  ASSISTANT: (uses full context above)           │  ← Informed by all
└─────────────────────────────────────────────────┘
```

### Experiment 4A: Full Pipeline — Code Review Assistant

In [13]:
messages = [
    {
        "role": "system",
        "content": (
            "You are a senior Python code reviewer. "
            "For each review, provide: "
            "1) A brief summary, "
            "2) Issues found (if any), "
            "3) Suggested improvements. "
            "Be constructive and concise."
        )
    },
    {
        "role": "user",
        "content": """Review this Python function:

def calc(x, y, z):
    result = x + y
    result = result * z
    if result > 100:
        return "big"
    else:
        return "small"
"""
    }
]

print("CODE REVIEW ASSISTANT")
print("=" * 60)
show_messages(messages)
_ = chat(messages)

CODE REVIEW ASSISTANT


### Summary

The provided `calc` function performs two operations on the inputs `x`, `y`, and `z`. The first is to add `x` and `y`, then multiply the result by `z`. It returns a string based on whether the final result exceeds 100. 

### Issues Found

- **Unused Parameter:**
  - The parameter `z` is not used in the function, so it can be removed for clarity.

- **Unnecessary Return Statement Inside If:**
  - Since the function only checks if `result > 100`, and that's always true given the current implementation, you could simplify this by removing the unnecessary `else` clause entirely.

### Suggested Improvements

```python
def calc(x, y):
    result = x + y
    result *= z
    return "big" if result > 100 else "small"
```

In summary, removing the unused parameter and combining the conditional logic into a single statement improves readability and simplifies the function.


⏱️ Response time: 2.18s


### Experiment 4B: Comparing System Prompt Strictness

How does the **level of detail** in a system prompt affect the model's behavior?

In [14]:
user_msg = "What are the benefits of exercise?"

system_levels = [
    {
        "label": "Minimal",
        "prompt": "You are a helpful assistant."
    },
    {
        "label": "Moderate",
        "prompt": "You are a health expert. Give practical advice in bullet points. Limit to 5 bullets."
    },
    {
        "label": "Strict",
        "prompt": (
            "You are a certified fitness coach. "
            "Rules: "
            "1) Always respond with exactly 3 bullet points. "
            "2) Each bullet must be one sentence only. "
            "3) Include a specific scientific benefit in each. "
            "4) End with a motivational one-liner."
        )
    }
]

for level in system_levels:
    print(f"\n{'=' * 60}")
    print(f"SYSTEM PROMPT LEVEL: {level['label']}")
    print(f"{'=' * 60}")
    messages = [
        {"role": "system", "content": level["prompt"]},
        {"role": "user", "content": user_msg}
    ]
    show_messages(messages)
    _ = chat(messages)


SYSTEM PROMPT LEVEL: Minimal


There are many benefits to regular physical activity, including:

1. Improved cardiovascular health: Regular exercise can help improve your heart and lung function, which in turn can reduce the risk of developing chronic diseases such as high blood pressure, stroke, and heart disease.

2. Weight management: Exercise is an effective way to burn calories and lose weight. It helps you maintain a healthy bodyweight and reduces the likelihood of obesity-related health problems like diabetes, heart disease, and certain types of cancer.

3. Improved mental health: Physical activity has been shown to reduce symptoms of depression, anxiety, and other mental health issues. Exercise can help improve self-esteem, mood, and overall well-being.

4. Increased energy levels: Regular exercise can boost your energy levels and make you feel more alert throughout the day. It helps regulate sleep patterns and reduces fatigue during the workday or evening activities.

5. Better posture and reduced risk of injury: Physical activity strengthens muscles and bones in the back, legs, and core, which helps improve posture and reduce the risk of injuries such as back pain, knee problems, and neck pain.

6. Improved sleep quality: Exercise can help you fall asleep faster and have a deeper sleep during the night. It also reduces insomnia symptoms like tossing and turning or waking up frequently throughout the night.

7. Increased longevity: Regular exercise has been linked to increased life expectancy and improved overall health in older adults, especially those with chronic diseases such as heart disease and diabetes.

Overall, regular physical activity can have a positive impact on your mental and physical well-being, improving quality of life and reducing the risk of many serious illnesses.


⏱️ Response time: 3.06s

SYSTEM PROMPT LEVEL: Moderate


- Reduces risk of chronic diseases like heart disease, diabetes and certain types of cancer.
- Improves mood and mental health by reducing symptoms of depression and anxiety.
- Enhances physical fitness and strength for better performance in daily activities.
- Boosts energy levels and improves overall quality of life.


⏱️ Response time: 0.65s

SYSTEM PROMPT LEVEL: Strict


- Exercise boosts cardiovascular health, reducing the risk of heart disease.
- Physical activity enhances mental well-being by decreasing symptoms of depression and anxiety.
- Engaging in regular physical exercise can improve sleep quality significantly.
- It helps maintain a healthy weight, preventing obesity-related issues such as type 2 diabetes.


⏱️ Response time: 0.68s


---

## Key Takeaways

| Concept | What to Remember |
|---------|------------------|
| **System Prompt** | Sets the "personality" and rules — processed first, shapes all responses |
| **User Prompt** | Your direct input — be specific for better results |
| **Assistant Role** | Model responses that form conversation memory |
| **Multi-turn** | Including prior assistant messages gives the model conversation context |
| **Specificity** | More detailed system prompts = more controlled outputs |
| **Conflicts** | System prompts generally take priority, but results vary by model |
| **Guardrails** | Use system prompts to constrain the model's scope and behavior |